In [2]:
import json
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from airflow.models import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.sqlite_operator import SqliteOperator
from sqlite_cli import SqLiteClient

BASE_URL = 'https://www.alphavantage.co/query'
API_KEY = 'TFHNYCWBD71JBSON'
STOCK_FN = 'TIME_SERIES_DAILY'

SQL_DB = '/tmp/sqlite_default.db'  # This is defined in Admin/Connections
SQL_TABLE = 'stocks_daily'
SQL_CREATE = f"""
CREATE TABLE IF NOT EXISTS {SQL_TABLE} (
date TEXT,
symbol TEXT,
avg_num_trades REAL,
avg_price REAL,
UNIQUE(date,symbol)
)
"""


def _get_stock_data(stock_symbol, **context):
    #date = f"{context['execution_date']:%Y-%m-%d}"  # read execution date from context
    end_point = (
        f"{BASE_URL}?function={STOCK_FN}&symbol={stock_symbol}"
        f"&apikey={API_KEY}&datatype=json"
    )
    print(f"Getting data from {end_point}...")
    r = requests.get(end_point)
    data = json.loads(r.content)
    print(data)
    df = (
        pd.DataFrame(data['Time Series (Daily)'])
        .T.reset_index()
        .rename(columns={'index': 'date'})
    )
    df = df[df['date'] == date]
    if not df.empty:
        for c in df.columns:
            if c != 'date':
                df[c] = df[c].astype(float)
        df['avg_price'] = (df['2. high'] + df['3. low']) / 2
        df['avg_num_trades'] = df['5. volume'] / 1440
    else:
        df = pd.DataFrame(
            [[date, np.nan, np.nan]],
            columns=['date', 'avg_num_trades', 'avg_price'],
        )
    df['symbol'] = stock_symbol
    df = df[['date', 'symbol', 'avg_num_trades', 'avg_price']]
    return df

print(df)

NameError: name 'df' is not defined